<a href="https://colab.research.google.com/github/brendachaka/FUTURE_CS_01/blob/main/Brenda_Chaka_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "transformers==4.40.0" datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [7]:
dataset = load_dataset("dair-ai/emotion")
dataset


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [8]:
id2label = {i: name for i, name in enumerate(dataset["train"].features["label"].names)}
label2id = {v: k for k, v in id2label.items()}

id2label, label2id


({0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'},
 {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5})

In [9]:
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )


In [11]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

encoded_dataset = encoded_dataset.remove_columns(["text"])
encoded_dataset.set_format("torch")

encoded_dataset


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [1]:
!nvidia-smi


Tue Nov 25 06:39:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
from torch.utils.data import DataLoader

# 1. Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Training on:", device)
model.to(device)

# 2. Create data loaders
train_dataloader = DataLoader(encoded_dataset["train"], batch_size=16, shuffle=True)
val_dataloader = DataLoader(encoded_dataset["validation"], batch_size=16)

# 3. Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_dataloader:
        # Move to GPU/CPU
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        # Model expects "labels", not "label"
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Training loss: {avg_loss:.4f}")


Training on: cuda
Epoch 1/2 - Training loss: 0.4430
Epoch 2/2 - Training loss: 0.1449


In [14]:
save_path = "/content/distilbert_emotion_model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved to:", save_path)


Model saved to: /content/distilbert_emotion_model


In [15]:
!zip -r distilbert_emotion_model.zip distilbert_emotion_model


  adding: distilbert_emotion_model/ (stored 0%)
  adding: distilbert_emotion_model/tokenizer_config.json (deflated 75%)
  adding: distilbert_emotion_model/model.safetensors (deflated 8%)
  adding: distilbert_emotion_model/config.json (deflated 49%)
  adding: distilbert_emotion_model/tokenizer.json (deflated 71%)
  adding: distilbert_emotion_model/special_tokens_map.json (deflated 42%)
  adding: distilbert_emotion_model/vocab.txt (deflated 53%)


In [16]:
!zip -r distilbert_emotion_model.zip distilbert_emotion_model


updating: distilbert_emotion_model/ (stored 0%)
updating: distilbert_emotion_model/tokenizer_config.json (deflated 75%)
updating: distilbert_emotion_model/model.safetensors (deflated 8%)
updating: distilbert_emotion_model/config.json (deflated 49%)
updating: distilbert_emotion_model/tokenizer.json (deflated 71%)
updating: distilbert_emotion_model/special_tokens_map.json (deflated 42%)
updating: distilbert_emotion_model/vocab.txt (deflated 53%)
